# Phiên 4 – So sánh SLM và LLM

So sánh độ trễ và chất lượng phản hồi mẫu giữa Mô hình Ngôn ngữ Nhỏ và mô hình lớn hơn chạy qua Foundry Local.


## ⚡ Bắt đầu nhanh

**Cài đặt tối ưu hóa bộ nhớ (Cập nhật):**
1. Các mô hình tự động chọn biến thể CPU (hoạt động trên mọi phần cứng)
2. Sử dụng `qwen2.5-3b` thay vì 7B (tiết kiệm ~4GB RAM)
3. Tự động phát hiện cổng (không cần cấu hình thủ công)
4. Tổng RAM cần thiết: ~8GB được khuyến nghị (bao gồm mô hình + hệ điều hành)

**Cài đặt Terminal (30 giây):**
```bash
foundry service start
foundry model run phi-4-mini
foundry model run qwen2.5-3b
```

Sau đó chạy notebook này! 🚀


### Giải thích: Cài đặt phụ thuộc
Cài đặt các gói tối thiểu (`foundry-local-sdk`, `openai`, `numpy`) cần thiết cho việc đo thời gian và yêu cầu trò chuyện. Có thể chạy lại một cách an toàn mà không gây ảnh hưởng.


# Kịch bản
So sánh một Mô hình Ngôn ngữ Nhỏ (SLM) đại diện với một mô hình lớn hơn trên một lời nhắc duy nhất để minh họa các đánh đổi:
- **Sự khác biệt về độ trễ** (thời gian thực tế)
- **Sử dụng token** (nếu có) làm đại diện cho thông lượng
- **Mẫu đầu ra định tính** để đánh giá nhanh
- **Tính toán tốc độ** để định lượng lợi ích hiệu suất

**Biến môi trường:**
- `SLM_ALIAS` - Mô hình ngôn ngữ nhỏ (mặc định: phi-4-mini, ~4GB RAM)
- `LLM_ALIAS` - Mô hình ngôn ngữ lớn hơn (mặc định: qwen2.5-7b, ~7GB RAM)
- `COMPARE_PROMPT` - Lời nhắc thử nghiệm để so sánh
- `COMPARE_RETRIES` - Số lần thử lại để tăng độ bền (mặc định: 2)
- `FOUNDRY_LOCAL_ENDPOINT` - Ghi đè điểm cuối dịch vụ (tự động phát hiện nếu không được đặt)

**Cách hoạt động (Mẫu SDK chính thức):**
1. **FoundryLocalManager** khởi tạo và quản lý dịch vụ Foundry Local
2. Dịch vụ tự động khởi động nếu chưa chạy (không cần thiết lập thủ công)
3. Các mô hình được giải quyết từ bí danh sang ID cụ thể một cách tự động
4. Các biến thể tối ưu hóa phần cứng được chọn (CUDA, NPU hoặc CPU)
5. Client tương thích OpenAI thực hiện hoàn thành cuộc trò chuyện
6. Các số liệu được thu thập: độ trễ, token, chất lượng đầu ra
7. Kết quả được so sánh để tính tỷ lệ tăng tốc

So sánh nhỏ này giúp quyết định khi nào nên chuyển hướng sang mô hình lớn hơn cho trường hợp sử dụng của bạn.

**Tham khảo SDK:** 
- Python SDK: https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local
- Workshop Utils: Sử dụng mẫu chính thức từ ../samples/workshop_utils.py

**Lợi ích chính:**
- ✅ Tự động phát hiện và khởi tạo dịch vụ
- ✅ Tự động khởi động dịch vụ nếu chưa chạy
- ✅ Tích hợp giải quyết mô hình và bộ nhớ đệm
- ✅ Tối ưu hóa phần cứng (CUDA/NPU/CPU)
- ✅ Tương thích SDK OpenAI
- ✅ Xử lý lỗi mạnh mẽ với các lần thử lại
- ✅ Suy luận cục bộ (không cần API đám mây)


## 🚨 Yêu cầu: Foundry Local phải đang chạy!

**Trước khi chạy notebook này**, hãy đảm bảo dịch vụ Foundry Local đã được thiết lập:

### Lệnh khởi động nhanh (Chạy trong Terminal):

```bash
# 1. Start the Foundry Local service
foundry service start

# 2. Load the default models used in this comparison (CPU-optimized)
foundry model run phi-4-mini
foundry model run qwen2.5-3b

# 3. Verify models are loaded
foundry model ls

# 4. Check service health
foundry service status
```

### Các mô hình thay thế (nếu không có mô hình mặc định):

```bash
# Even smaller alternatives (if memory is very limited)
foundry model run phi-3.5-mini
foundry model run qwen2.5-0.5b

# Or update the environment variables in this notebook:
# SLM_ALIAS = 'phi-3.5-mini'
# LLM_ALIAS = 'qwen2.5-1.5b'  # Or qwen2.5-0.5b for minimum memory
```

⚠️ **Nếu bạn bỏ qua các bước này**, bạn sẽ gặp lỗi `APIConnectionError` khi chạy các ô notebook bên dưới.


In [29]:
# Install dependencies
!pip install -q foundry-local-sdk openai numpy requests

### Giải thích: Nhập lõi
Đưa vào các tiện ích về thời gian và các khách hàng Foundry Local / OpenAI được sử dụng để lấy thông tin mô hình và thực hiện hoàn thành trò chuyện.


In [30]:
import os, time, json
from foundry_local import FoundryLocalManager
from openai import OpenAI
import sys
sys.path.append('../samples')
from workshop_utils import get_client, chat_once

### Giải thích: Bí danh & Cài đặt Lời nhắc
Định nghĩa các bí danh có thể cấu hình theo môi trường cho mô hình nhỏ và lớn hơn cùng với một lời nhắc so sánh. Điều chỉnh các biến môi trường để thử nghiệm với các họ mô hình hoặc nhiệm vụ khác nhau.


In [31]:
# Default to CPU models for better memory efficiency
SLM = os.getenv('SLM_ALIAS', 'phi-4-mini')  # Auto-selects CPU variant
LLM = os.getenv('LLM_ALIAS', 'qwen2.5-3b')  # Smaller LLM, more memory-friendly
PROMPT = os.getenv('COMPARE_PROMPT', 'List 5 benefits of local AI inference.')
# Endpoint is now managed by FoundryLocalManager - it auto-detects or can be overridden
ENDPOINT = os.getenv('FOUNDRY_LOCAL_ENDPOINT', None)

### 💡 Cấu hình tối ưu hóa bộ nhớ

**Notebook này sử dụng các mô hình tiết kiệm bộ nhớ theo mặc định:**
- `phi-4-mini` → ~4GB RAM (Foundry Local tự động chọn biến thể CPU)
- `qwen2.5-3b` → ~3GB RAM (thay vì 7B cần ~7GB+)

**Tự động phát hiện cổng:**
- Foundry Local có thể sử dụng các cổng khác nhau (thường là 55769 hoặc 59959)
- Ô chẩn đoán bên dưới sẽ tự động phát hiện cổng chính xác
- Không cần cấu hình thủ công!

**Nếu bạn có RAM hạn chế (<8GB), hãy sử dụng các mô hình nhỏ hơn nữa:**
```python
SLM = 'phi-3.5-mini'      # ~2GB
LLM = 'qwen2.5-0.5b'      # ~500MB
```


In [32]:
# Display current configuration
print("="*60)
print("CURRENT CONFIGURATION")
print("="*60)
print(f"SLM Model:     {SLM}")
print(f"LLM Model:     {LLM}")
print(f"SDK Pattern:   FoundryLocalManager (official)")
print(f"Endpoint:      {ENDPOINT or 'Auto-detect'}")
print(f"Test Prompt:   {PROMPT[:50]}...")
print(f"Retry Count:   2")
print("="*60)
print("\n💡 Using official Foundry SDK pattern from workshop_utils")
print("   → FoundryLocalManager handles service lifecycle")
print("   → Automatic model resolution and hardware optimization")
print("   → OpenAI-compatible API for inference")

CURRENT CONFIGURATION
SLM Model:     phi-4-mini
LLM Model:     qwen2.5-7b
SDK Pattern:   FoundryLocalManager (official)
Endpoint:      Auto-detect
Test Prompt:   List 5 benefits of local AI inference....
Retry Count:   2

💡 Using official Foundry SDK pattern from workshop_utils
   → FoundryLocalManager handles service lifecycle
   → Automatic model resolution and hardware optimization
   → OpenAI-compatible API for inference


### Giải thích: Công cụ hỗ trợ thực thi (Mẫu SDK Foundry)
Sử dụng mẫu SDK Foundry Local chính thức như đã được tài liệu hóa trong các ví dụ của Workshop:

**Cách tiếp cận:**
- **FoundryLocalManager** - Khởi tạo và quản lý dịch vụ Foundry Local
- **Tự động phát hiện** - Tự động tìm kiếm điểm cuối và xử lý vòng đời dịch vụ
- **Giải quyết mô hình** - Chuyển đổi bí danh thành ID mô hình đầy đủ (ví dụ: phi-4-mini → phi-4-mini-instruct-cpu)
- **Tối ưu hóa phần cứng** - Lựa chọn biến thể tốt nhất cho phần cứng hiện có (CUDA, NPU, hoặc CPU)
- **OpenAI Client** - Được cấu hình với điểm cuối của trình quản lý để truy cập API tương thích với OpenAI

**Tính năng chịu lỗi:**
- Logic thử lại với thời gian chờ tăng dần (có thể cấu hình qua môi trường)
- Tự động khởi động dịch vụ nếu chưa chạy
- Xác minh kết nối sau khi khởi tạo
- Xử lý lỗi một cách nhẹ nhàng với báo cáo lỗi chi tiết
- Bộ nhớ đệm mô hình để tránh khởi tạo lặp lại

**Cấu trúc kết quả:**
- Đo lường độ trễ (thời gian thực)
- Theo dõi việc sử dụng token (nếu có)
- Kết quả mẫu (được rút gọn để dễ đọc)
- Chi tiết lỗi cho các yêu cầu thất bại

Mẫu này tận dụng module workshop_utils, tuân theo mẫu SDK chính thức.

**Tham khảo SDK:**
- Kho chính: https://github.com/microsoft/Foundry-Local
- Python SDK: https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local
- Workshop Utils: ../samples/workshop_utils.py


In [39]:
def setup(alias: str, endpoint: str = None, retries: int = 3):
    """
    Initialize a Foundry Local model connection using official SDK pattern.
    
    This follows the workshop_utils pattern which uses FoundryLocalManager
    to properly initialize the Foundry Local service and resolve models.
    
    Args:
        alias: Model alias (e.g., 'phi-4-mini', 'qwen2.5-3b')
        endpoint: Optional endpoint override (usually auto-detected)
        retries: Number of connection attempts (default: 3)
    
    Returns:
        tuple: (manager, client, model_id, metadata) or (None, None, alias, error_metadata) if failed
    """
    import time
    
    last_err = None
    current_delay = 2  # seconds
    
    for attempt in range(1, retries + 1):
        try:
            print(f"[Init] Connecting to '{alias}' (attempt {attempt}/{retries})...")
            
            # Use the workshop utility which follows the official SDK pattern
            manager, client, model_id = get_client(alias, endpoint=endpoint)
            
            print(f"[OK] Connected to '{alias}' -> {model_id}")
            print(f"     Endpoint: {manager.endpoint}")
            
            return manager, client, model_id, {
                'endpoint': manager.endpoint,
                'resolved': model_id,
                'attempts': attempt,
                'status': 'success'
            }
            
        except Exception as e:
            last_err = e
            error_msg = str(e)
            
            # Provide helpful error messages
            if "Connection error" in error_msg or "connection refused" in error_msg.lower():
                print(f"[ERROR] Cannot connect to Foundry Local service")
                print(f"        → Is the service running? Try: foundry service start")
                print(f"        → Is the model loaded? Try: foundry model run {alias}")
            elif "not found" in error_msg.lower():
                print(f"[ERROR] Model '{alias}' not found in catalog")
                print(f"        → Available models: Run 'foundry model ls' in terminal")
                print(f"        → Download model: Run 'foundry model download {alias}'")
            else:
                print(f"[ERROR] Setup failed: {type(e).__name__}: {error_msg}")
            
            if attempt < retries:
                print(f"[Retry] Waiting {current_delay:.1f}s before retry...")
                time.sleep(current_delay)
                current_delay *= 2  # Exponential backoff
    
    # All retries failed - provide actionable guidance
    print(f"\n❌ Failed to initialize '{alias}' after {retries} attempts")
    print(f"   Last error: {type(last_err).__name__}: {str(last_err)}")
    print(f"\n💡 Troubleshooting steps:")
    print(f"   1. Ensure Foundry Local service is running:")
    print(f"      → foundry service status")
    print(f"      → foundry service start (if not running)")
    print(f"   2. Ensure model is loaded:")
    print(f"      → foundry model run {alias}")
    print(f"   3. Check available models:")
    print(f"      → foundry model ls")
    print(f"   4. Try alternative models if '{alias}' isn't available")
    
    return None, None, alias, {
        'error': f"{type(last_err).__name__}: {str(last_err)}",
        'endpoint': endpoint or 'auto-detect',
        'attempts': retries,
        'status': 'failed'
    }


def run(client, model_id: str, prompt: str, max_tokens: int = 180, temperature: float = 0.5):
    """
    Run inference with the configured model using OpenAI SDK.
    
    Args:
        client: OpenAI client instance (configured for Foundry Local)
        model_id: Model identifier (resolved from alias)
        prompt: Input prompt
        max_tokens: Maximum response tokens (default: 180)
        temperature: Sampling temperature (default: 0.5)
    
    Returns:
        dict: Response with timing, tokens, and content
    """
    import time
    
    start = time.time()
    
    try:
        response = client.chat.completions.create(
            model=model_id,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=temperature
        )
        
        elapsed = time.time() - start
        
        # Extract response details
        content = response.choices[0].message.content
        
        # Try to extract token usage from multiple possible locations
        usage_info = {}
        if hasattr(response, 'usage') and response.usage:
            usage_info['prompt_tokens'] = getattr(response.usage, 'prompt_tokens', None)
            usage_info['completion_tokens'] = getattr(response.usage, 'completion_tokens', None)
            usage_info['total_tokens'] = getattr(response.usage, 'total_tokens', None)
        
        # Calculate approximate token count if API doesn't provide it
        # Rough estimate: ~4 characters per token for English text
        if not usage_info.get('total_tokens'):
            estimated_prompt_tokens = len(prompt) // 4
            estimated_completion_tokens = len(content) // 4
            estimated_total = estimated_prompt_tokens + estimated_completion_tokens
            usage_info['estimated_tokens'] = estimated_total
            usage_info['estimated_prompt_tokens'] = estimated_prompt_tokens
            usage_info['estimated_completion_tokens'] = estimated_completion_tokens
        
        return {
            'status': 'success',
            'content': content,
            'elapsed_sec': elapsed,
            'tokens': usage_info.get('total_tokens') or usage_info.get('estimated_tokens'),
            'usage': usage_info,
            'model': model_id
        }
        
    except Exception as e:
        elapsed = time.time() - start
        return {
            'status': 'error',
            'error': f"{type(e).__name__}: {str(e)}",
            'elapsed_sec': elapsed,
            'model': model_id
        }


print("✅ Execution helpers defined: setup(), run()")
print("   → Uses workshop_utils for proper SDK integration")
print("   → setup() initializes with FoundryLocalManager")
print("   → run() executes inference via OpenAI-compatible API")
print("   → Token counting: Uses API data or estimates if unavailable")

✅ Execution helpers defined: setup(), run()
   → Uses workshop_utils for proper SDK integration
   → setup() initializes with FoundryLocalManager
   → run() executes inference via OpenAI-compatible API
   → Token counting: Uses API data or estimates if unavailable


### Giải thích: Kiểm tra tự động trước khi chạy
Thực hiện kiểm tra kết nối nhẹ bằng FoundryLocalManager cho cả hai mô hình. Điều này xác minh:
- Dịch vụ có thể truy cập được
- Các mô hình có thể được khởi tạo
- Các bí danh được giải quyết thành ID mô hình thực tế
- Kết nối ổn định trước khi chạy so sánh

Hàm setup() sử dụng mẫu SDK chính thức từ workshop_utils.


In [34]:
# Simplified diagnostic: Just verify service is accessible
import requests

def check_foundry_service():
    """Quick diagnostic to verify Foundry Local is running."""
    # Try common ports
    endpoints_to_try = [
        "http://localhost:59959",
        "http://127.0.0.1:59959", 
        "http://localhost:55769",
        "http://127.0.0.1:55769",
    ]
    
    print("[Diagnostic] Checking Foundry Local service...")
    
    for endpoint in endpoints_to_try:
        try:
            response = requests.get(f"{endpoint}/health", timeout=2)
            if response.status_code == 200:
                print(f"✅ Service is running at {endpoint}")
                
                # Try to list models
                try:
                    models_response = requests.get(f"{endpoint}/v1/models", timeout=2)
                    if models_response.status_code == 200:
                        models_data = models_response.json()
                        model_count = len(models_data.get('data', []))
                        print(f"✅ Found {model_count} models available")
                        if model_count > 0:
                            print("   Models:", [m.get('id', 'unknown') for m in models_data.get('data', [])[:5]])
                except Exception as e:
                    print(f"⚠️  Could not list models: {e}")
                
                return endpoint
        except requests.exceptions.ConnectionError:
            continue
        except Exception as e:
            print(f"⚠️  Error checking {endpoint}: {e}")
    
    print("\n❌ Foundry Local service not found!")
    print("\n💡 To fix this:")
    print("   1. Open a terminal")
    print("   2. Run: foundry service start")
    print("   3. Run: foundry model run phi-4-mini")
    print("   4. Run: foundry model run qwen2.5-3b")
    print("   5. Re-run this notebook")
    return None

# Run diagnostic
discovered_endpoint = check_foundry_service()

if discovered_endpoint:
    print(f"\n✅ Service detected (will be managed by FoundryLocalManager)")
else:
    print(f"\n⚠️  No service detected - FoundryLocalManager will attempt to start it")

[Diagnostic] Checking Foundry Local service...

❌ Foundry Local service not found!

💡 To fix this:
   1. Open a terminal
   2. Run: foundry service start
   3. Run: foundry model run phi-4-mini
   4. Run: foundry model run qwen2.5-3b
   5. Re-run this notebook

⚠️  No service detected - FoundryLocalManager will attempt to start it


In [35]:
# Quick Fix: Start service and load models from notebook
# Uncomment the commands you need:

# !foundry service start
# !foundry model run phi-4-mini
# !foundry model run qwen2.5-3b
# !foundry model ls

print("⚠️  The commands above are commented out.")
print("Uncomment them if you want to start the service from the notebook.")
print("")
print("💡 Recommended: Run these commands in a separate terminal instead:")
print("   foundry service start")
print("   foundry model run phi-4-mini")
print("   foundry model run qwen2.5-3b")

⚠️  The commands above are commented out.
Uncomment them if you want to start the service from the notebook.

💡 Recommended: Run these commands in a separate terminal instead:
   foundry service start
   foundry model run phi-4-mini
   foundry model run qwen2.5-3b


### 🛠️ Sửa nhanh: Khởi động Foundry Local từ Notebook (Tùy chọn)

Nếu chẩn đoán ở trên cho thấy dịch vụ không hoạt động, bạn có thể thử khởi động nó từ đây:

**Lưu ý:** Điều này hoạt động tốt nhất trên Windows. Trên các nền tảng khác, hãy sử dụng lệnh trong terminal.


### ⚠️ Khắc phục lỗi kết nối

Nếu bạn thấy `APIConnectionError`, dịch vụ Foundry Local có thể không đang chạy hoặc các mô hình chưa được tải. Hãy thử các bước sau:

**1. Kiểm tra trạng thái dịch vụ:**
```bash
# In a terminal (not in notebook):
foundry service status
```

**2. Khởi động dịch vụ (nếu chưa chạy):**
```bash
foundry service start
```

**3. Tải các mô hình cần thiết:**
```bash
# Load the models needed for comparison
foundry model run phi-4-mini
foundry model run qwen2.5-7b

# Or use alternative models:
foundry model run phi-3.5-mini
foundry model run qwen2.5-3b
```

**4. Xác minh các mô hình đã sẵn sàng:**
```bash
foundry model ls
```

**Các vấn đề thường gặp:**
- ❌ Dịch vụ không chạy → Chạy `foundry service start`
- ❌ Mô hình chưa được tải → Chạy `foundry model run <model-name>`
- ❌ Xung đột cổng → Kiểm tra xem có dịch vụ nào khác đang sử dụng cổng
- ❌ Tường lửa chặn → Đảm bảo kết nối nội bộ được phép

**Sửa nhanh:** Chạy ô chẩn đoán dưới đây trước khi kiểm tra tiền khởi động.


In [36]:
preflight = {}
retries = 2  # Number of retry attempts

for a in (SLM, LLM):
    mgr, c, mid, info = setup(a, endpoint=ENDPOINT, retries=retries)
    # Keep the original status from info (either 'success' or 'failed')
    preflight[a] = info

print('\n[Pre-flight Check]')
for alias, details in preflight.items():
    status_icon = '✅' if details['status'] == 'success' else '❌'
    print(f"  {status_icon} {alias}: {details['status']} - {details.get('resolved', details.get('error', 'unknown'))}")

preflight

[Init] Connecting to 'phi-4-mini' (attempt 1/2)...
[OK] Connected to 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1

[Pre-flight Check]
  ✅ phi-4-mini: success - Phi-4-mini-instruct-cuda-gpu:4
  ✅ qwen2.5-7b: success - qwen2.5-7b-instruct-cuda-gpu:3


{'phi-4-mini': {'endpoint': 'http://127.0.0.1:59959/v1',
  'resolved': 'Phi-4-mini-instruct-cuda-gpu:4',
  'attempts': 1,
  'status': 'success'},
 'qwen2.5-7b': {'endpoint': 'http://127.0.0.1:59959/v1',
  'resolved': 'qwen2.5-7b-instruct-cuda-gpu:3',
  'attempts': 1,
  'status': 'success'}}

### ✅ Kiểm tra trước khi chạy: Khả dụng của mô hình

Phần này kiểm tra xem cả hai mô hình có thể được truy cập tại điểm cuối đã cấu hình trước khi thực hiện so sánh.


### Giải thích: So sánh Chạy & Thu Thập Kết Quả
Lặp qua cả hai bí danh bằng cách sử dụng mẫu chính thức của Foundry SDK:
1. Khởi tạo từng mô hình với setup() (sử dụng FoundryLocalManager)
2. Thực hiện suy luận với API tương thích OpenAI
3. Ghi lại độ trễ, số lượng token và mẫu đầu ra
4. Tạo bản tóm tắt JSON với phân tích so sánh

Điều này tuân theo cùng một mẫu như các ví dụ trong Workshop tại session04/model_compare.py.


In [40]:
results = []
retries = 2  # Number of retry attempts

for alias in (SLM, LLM):
    mgr, client, mid, info = setup(alias, endpoint=ENDPOINT, retries=retries)
    if client:
        r = run(client, mid, PROMPT)
        results.append({'alias': alias, **r})
    else:
        # If setup failed, record error
        results.append({
            'alias': alias,
            'status': 'error',
            'error': info.get('error', 'Setup failed'),
            'elapsed_sec': 0,
            'tokens': None,
            'model': alias
        })

# Display results
print(json.dumps(results, indent=2))

# Quick comparative view
print('\n' + '='*80)
print('COMPARISON SUMMARY')
print('='*80)
print(f"{'Alias':<20} {'Status':<15} {'Latency(s)':<15} {'Tokens':<15}")
print('-'*80)

for row in results:
    status = row.get('status', 'unknown')
    status_icon = '✅' if status == 'success' else '❌'
    latency_str = f"{row.get('elapsed_sec', 0):.3f}" if row.get('elapsed_sec') else 'N/A'
    
    # Handle token display - show if available or indicate estimated
    tokens = row.get('tokens')
    usage = row.get('usage', {})
    if tokens:
        if 'estimated_tokens' in usage:
            tokens_str = f"~{tokens} (est.)"
        else:
            tokens_str = str(tokens)
    else:
        tokens_str = 'N/A'
    
    print(f"{status_icon} {row['alias']:<18} {status:<15} {latency_str:<15} {tokens_str:<15}")

print('-'*80)

# Show detailed token breakdown if available
print("\nDetailed Token Usage:")
for row in results:
    if row.get('status') == 'success' and row.get('usage'):
        usage = row['usage']
        print(f"\n  {row['alias']}:")
        if 'prompt_tokens' in usage and usage['prompt_tokens']:
            print(f"    Prompt tokens:     {usage['prompt_tokens']}")
            print(f"    Completion tokens: {usage['completion_tokens']}")
            print(f"    Total tokens:      {usage['total_tokens']}")
        elif 'estimated_tokens' in usage:
            print(f"    Estimated prompt:     {usage['estimated_prompt_tokens']}")
            print(f"    Estimated completion: {usage['estimated_completion_tokens']}")
            print(f"    Estimated total:      {usage['estimated_tokens']}")
            print(f"    (API did not provide token counts - using ~4 chars/token estimate)")

print('\n' + '='*80)

# Calculate speedup if both succeeded
if len(results) == 2 and all(r.get('status') == 'success' and r.get('elapsed_sec') for r in results):
    speedup = results[1]['elapsed_sec'] / results[0]['elapsed_sec']
    print(f"\n💡 SLM is {speedup:.2f}x faster than LLM for this prompt")
    
    # Compare token throughput if available
    slm_tokens = results[0].get('tokens', 0)
    llm_tokens = results[1].get('tokens', 0)
    if slm_tokens and llm_tokens:
        slm_tps = slm_tokens / results[0]['elapsed_sec']
        llm_tps = llm_tokens / results[1]['elapsed_sec']
        print(f"   SLM throughput: {slm_tps:.1f} tokens/sec")
        print(f"   LLM throughput: {llm_tps:.1f} tokens/sec")
        
elif any(r.get('status') == 'error' for r in results):
    print(f"\n⚠️  Some models failed - check errors above")
    print("   Ensure Foundry Local is running: foundry service start")
    print("   Ensure models are loaded: foundry model run <model-name>")

results

[Init] Connecting to 'phi-4-mini' (attempt 1/2)...
[OK] Connected to 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1
[
  {
    "alias": "phi-4-mini",
    "status": "success",
    "content": "1. Reduced Latency: Local AI inference can significantly reduce latency by processing data closer to the source, which is particularly beneficial for real-time applications such as autonomous vehicles or augmented reality.\n\n2. Enhanced Privacy: By keeping data processing local, sensitive information is less likely to be exposed to external networks, thereby enhancing privacy and security.\n\n3. Lower Bandwidth Usage: Local AI inference reduces the n

[{'alias': 'phi-4-mini',
  'status': 'success',
  'content': '1. Reduced Latency: Local AI inference can significantly reduce latency by processing data closer to the source, which is particularly beneficial for real-time applications such as autonomous vehicles or augmented reality.\n\n2. Enhanced Privacy: By keeping data processing local, sensitive information is less likely to be exposed to external networks, thereby enhancing privacy and security.\n\n3. Lower Bandwidth Usage: Local AI inference reduces the need for data transmission over the network, which can save bandwidth and reduce the risk of network congestion.\n\n4. Improved Reliability: Local processing can be more reliable, as it is less dependent on network connectivity. This is particularly important in scenarios where network connectivity is unreliable or intermittent.\n\n5. Scalability: Local AI inference can be easily scaled by adding more local processing units, making it easier to handle increasing data volumes or m

### Diễn giải Kết quả

**Các chỉ số chính:**
- **Độ trễ**: Càng thấp càng tốt - cho thấy thời gian phản hồi nhanh hơn
- **Tokens**: Lưu lượng cao hơn = xử lý được nhiều tokens hơn
- **Route**: Xác nhận API endpoint nào đã được sử dụng

**Khi nào nên dùng SLM và LLM:**
- **SLM (Mô hình ngôn ngữ nhỏ)**: Phản hồi nhanh, sử dụng ít tài nguyên, phù hợp với các nhiệm vụ đơn giản
- **LLM (Mô hình ngôn ngữ lớn)**: Chất lượng cao hơn, khả năng suy luận tốt hơn, sử dụng khi chất lượng là yếu tố quan trọng nhất

**Các bước tiếp theo:**
1. Thử các lời nhắc khác nhau để xem độ phức tạp ảnh hưởng thế nào đến sự so sánh
2. Thử nghiệm với các cặp mô hình khác
3. Sử dụng các mẫu định tuyến Workshop (Phiên 06) để định tuyến thông minh dựa trên độ phức tạp của nhiệm vụ


In [38]:
# Final Validation Check
print("="*70)
print("VALIDATION SUMMARY")
print("="*70)
print(f"✅ SLM Model: {SLM}")
print(f"✅ LLM Model: {LLM}")
print(f"✅ Using Foundry SDK Pattern: workshop_utils with FoundryLocalManager")
print(f"✅ Pre-flight passed: {all(v['status'] == 'success' for v in preflight.values()) if 'preflight' in dir() else 'Not run yet'}")
print(f"✅ Comparison completed: {len(results) == 2 if 'results' in dir() else 'Not run yet'}")
print(f"✅ Both models responded: {all(r.get('status') == 'success' for r in results) if 'results' in dir() and results else 'Not run yet'}")
print("="*70)

# Check for common configuration issues
issues = []
if 'LLM' in dir() and LLM not in ['qwen2.5-3b', 'qwen2.5-0.5b', 'qwen2.5-1.5b', 'qwen2.5-7b', 'phi-3.5-mini']:
    issues.append(f"⚠️  LLM is '{LLM}' - expected qwen2.5-3b for memory efficiency")
if 'preflight' in dir() and not all(v['status'] == 'success' for v in preflight.values()):
    issues.append("⚠️  Pre-flight check failed - models not accessible")
if 'results' in dir() and results and not all(r.get('status') == 'success' for r in results):
    issues.append("⚠️  Comparison incomplete - check for errors above")

if not issues and 'results' in dir() and results and all(r.get('status') == 'success' for r in results):
    print("🎉 ALL CHECKS PASSED! Notebook completed successfully.")
    print(f"   SLM ({SLM}) vs LLM ({LLM}) comparison completed.")
    if len(results) == 2:
        speedup = results[1]['elapsed_sec'] / results[0]['elapsed_sec'] if results[0]['elapsed_sec'] > 0 else 0
        print(f"   Performance: SLM is {speedup:.2f}x faster")
elif issues:
    print("\n⚠️  Issues detected:")
    for issue in issues:
        print(f"   {issue}")
    print("\n💡 Troubleshooting:")
    print("   1. Ensure service is running: foundry service start")
    print("   2. Load models: foundry model run phi-4-mini && foundry model run qwen2.5-7b")
    print("   3. Check model list: foundry model ls")
else:
    print("\n💡 Run all cells above first, then re-run this validation.")
print("="*70)

VALIDATION SUMMARY
✅ SLM Model: phi-4-mini
✅ LLM Model: qwen2.5-7b
✅ Using Foundry SDK Pattern: workshop_utils with FoundryLocalManager
✅ Pre-flight passed: True
✅ Comparison completed: True
✅ Both models responded: True
🎉 ALL CHECKS PASSED! Notebook completed successfully.
   SLM (phi-4-mini) vs LLM (qwen2.5-7b) comparison completed.
   Performance: SLM is 5.14x faster



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với các thông tin quan trọng, khuyến nghị sử dụng dịch vụ dịch thuật chuyên nghiệp bởi con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
